In [ ]:
# setup
from IPython.core.display import display,HTML
display(HTML('<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>'))
display(HTML(open('../rise.css').read()))

# imports
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.5, rc={'figure.figsize':(12, 6)})

<h1>Max Flow and Linear Programming</h1>

<h3>Last Time</h3>
We saw some examples of how probability theory can be useful in constructing algorithms. Before that, we saw transversal matroids and gave an algorithm for determining matchings in bipartite graphs.

<h3>This Time</h3>
We see another, more modern method to calculate matchings in bipartite graphs based on linear programming. We will then develop this method of linear programming and see that it is a general and useful technique. We will later apply it to the problem of drawing graphs.

<h3>Max Flow and Bipartite Matchings </h3>

Let $G=W\cup J$ be a bipartite graph. Recall that in order to solve the employment problem, we needed an algorithm to determine a subset of $W$ could be matched with a subset of $G$. We argued that we could search the graph for alternating paths. Finding an alternating path allowed us to extend the matching. This is essentially Boom's algorithm and it is nice because it is direct and combinatorial.

Now, we show that the problem can be cast easily as a max flow problem. In the max flow problem, we are given a directed graph with two special vertices, $source$ and $sink$. The edges have non-negative numbers called capacities. We want to assign numbers (a flow) to the edges such that each edge's flow is not more than its capacity, and, for each vertex, the sum of the flows into that vertex are equal to the flows out of that vertex. Finally, the solution to the max flow problem is the maximum amount of flow into the sink.

Here is a contruction that transforms our bipartite matching problem into a max flow problem: Orient the edges of $G$ from $W$ to $J$ and add two additional vertices $source$ and $sink$. Draw an edge from $source$ to each vertex of $W$ and an edge from each vertex of $J$ to $sink$. Assign the capacity of $1$ to each edge. It should be clear that each matching corresponds to a flow. Since flows do not need to be integers, it's less obvious that each flow can be transformed into a matching.

<img src="figures/max_flow.jpg" width="30%">



<h3>Linear programming</h3>

Let $n\in \mathbb{N}$ and let $c\in \mathbb{R}^n$ and $A$ be an $m\times n$ real-valued matrix. Let $b\in \mathbb{R}^m$. A linear program is a problem of the form:

Maximize $c^Tx$ such that $Ax\leq b$ and $x\geq 0$ where $x$ ranges over $\mathbb{R}^n$. Here, the inequalities are vector inequalities, meaning that they are required to hold in each coordinate.

Linear programs are significant because many problems (like max flow) can be cast as linear programs, and linear programs can be solved efficiently. For example, the linear program associated with the max flow problem sets $n=V(G)+E(G)$ and $m=E(G)+2V(G)$. We create $n$ variables, one for each edge of the flow graph. We get a constraint for each edge (that the flow cannot exceed the capacity) and two constraints for each vertex (the flow into a vertex is equal to the flow out; equality is two non-strict inequalities).

In [19]:
#ChatGPT Prompt: Write me a Python script to solve max flow of a bipartite graph using numpy and linear programming
#It's not so clear that this script is correct. TODO: Check this script carefully.
import numpy as np
from scipy.optimize import linprog

def max_flow_bipartite(adjacency_matrix,n,m):
    """
    Solves the maximum flow problem in a bipartite graph using linear programming.

    Parameters:
        adjacency_matrix (numpy.ndarray): Binary adjacency matrix of the bipartite graph.
                                           Shape: (n, m), where n is the number of left-side nodes
                                           and m is the number of right-side nodes.

    Returns:
        float: Maximum flow value.
    """

    # Total number of nodes: left-side + right-side + source + sink
    total_nodes = n + m + 2
    source = total_nodes - 2
    sink = total_nodes - 1

    # Total number of variables: flows along edges (source->left, left->right, right->sink)
    total_vars = n + m + adjacency_matrix.sum()

    # Objective: maximize the total flow into the sink
    c = np.zeros(total_vars)
    c[-m:] = -1  # Coefficients for flows into the sink (maximize by minimizing -flow)

    # Equality constraints (flow conservation):
    A_eq = []
    b_eq = []

    flow_var_idx = n + m  # Start index for left->right edge variables
    edge_map = {}

    # Source to left nodes
    for i in range(n):
        row = np.zeros(total_vars)
        row[i] = 1  # Flow from source to left node i
        A_eq.append(row)
        b_eq.append(1)  # Each left node gets at most 1 unit of flow

    # Left to right edges
    for i in range(n):
        for j in range(m):
            if adjacency_matrix[i, j] == 1:
                edge_map[(i, j)] = flow_var_idx
                row = np.zeros(total_vars)
                row[i] = -1  # Outflow from left node i
                row[flow_var_idx] = 1  # Inflow to edge (i, j)
                A_eq.append(row)
                b_eq.append(0)
                flow_var_idx += 1

    # Right nodes to sink
    for j in range(m):
        row = np.zeros(total_vars)
        row[n + j] = 1  # Flow into right node j
        for i in range(n):
            if (i, j) in edge_map:
                row[edge_map[(i, j)]] = -1  # Outflow from edge (i, j)
        A_eq.append(row)
        b_eq.append(1)  # Each right node gets at most 1 unit of flow

    # Combine into matrix form
    A_eq = np.array(A_eq)
    b_eq = np.array(b_eq)

    # Bounds for flow variables: All flows are non-negative and ≤ 1
    bounds = [(0, 1)] * total_vars

    # Solve the linear programming problem
    #print("linear programming parameters:", c, A_eq, b_eq, bounds)
    result = linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method="highs")

    if result.success:
        return -result.fun  # Return the maximum flow value
    else:
        raise ValueError("Linear programming failed to solve the problem.")

if __name__ == "__main__":
    # Example bipartite graph (adjacency matrix):
    adjacency_matrix = np.array([
        [0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 1, 0],
        [0, 0, 0, 0, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 1],
        [1, 1, 0, 0, 0, 0, 0],
        [0, 1, 1, 0, 0, 0, 0],
        [0, 0, 1, 1, 0, 0, 0]
    ])


    max_flow = max_flow_bipartite(adjacency_matrix,4,3)
    print(f"Maximum flow: {max_flow}")

Maximum flow: 3.0


<h3>Linear Programming Duality: Maxflow = Mincut</h3>

Every linear program can be equivalently handled as a dual linear program. When finding the maxflow, we could just as easily find a mincut- a set of edges with minimum weight whose removal disconnects the bipartite graph.

It's obvious that the max flow cannot exceed the minimum cut, since any flow will need to pass through the cut. The powerful theorem known as linear programming duality asserts that the two quantities are equal. More generally, linear programming duality asserts that any linear program has the same optimum as its dual.

If $c,A,b$ are the parameters to a linear program of the form: maximize $x\in \mathbb{R}^n$ such that $c^Tx$ is maximized subject to the constraints that $Ax\leq b$ and $x\geq 0$, then the dual linear program is defined to be the problem: minimize $b^Ty$ subject to the constraint $A^Ty\geq c$ and $y\geq 0$. The dual problem and the original problem have the same optimal values, according to the theorem of linear programming duality.

<h3>Economic interpretation of linear programming duality</h3>

Suppose that we have a car factory. We have several models that we build, and each model uses different parts. We have a certain number of each type of part in stock. Each model also sells for a different cost. Which cars should we build in order to maximize our revenue?

In more detail, the vector $c$ reflects the cost of each car. The vector $x$ represents the amount of each model that we should build. The matrix $A$ records how many of each part is needed for each model. The vector $b$ records how much of each part we have in stock.

The dual program can be interpreted as asking for the minimum amount that we should sell each part for in order to not incur an opportunity cost. The vector $y$ represents how much we will sell each part for. The vector $b$ still represents how many parts we have in stock. The equation $A^Ty \geq c$ reflects that we should sell each part more money that we could have made from making cars with those parts. The amazing statement of linear programming duality is that the minimal cost to sell each part for is equal to the maximum profit that we could make by actually making cars.

<h3>Algorithms for linear programming</h3>

Linear progamming can be solved efficiently in polynomial time. However, in practice, we usually use the [simplex method](https://en.wikipedia.org/wiki/Simplex_algorithm), which is theoretically a worst-case exponential algorithm. The algorithm operates by intepreting the "feasible region" where $Ax\leq b$ as a polytope. The algorithm systematically checks all of the vertices of this polytope. It is theoretically an exponential runtime because there are some instances where the polytope has exponentially many vertices. In particular, the hypercube has this property.

Linear programming is a special case of a more general technique, convex programming, where the feasible region is assumed to be convex. The Elipsoid method is a theoretically-polynomial time method that solves convex programming by repeatedly using smaller and smaller elipsoids to hone in on the optimal solution. This method is not practical, but it is of theoretical interest because it demonstrates that linear programming actually has a polynomial work solution.

A better technique for solving convex programming problems in polynomial time are known as <i>interior point methods</i>. These methods use the fact that the feasible region is convex to search a path through the interior of the feasible region for an optimal point. Detailed information on these methods can be found [here](https://github.com/ShiqinHuo/Numerical-Optimization-Books/blob/master/Convex%20Optimization%20Boyd.pdf).